# Download Data Section

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import datetime
import time


def grab_data(prov,station):
    
    url = fr'https://dd.weather.gc.ca/climate/ahccd/geojson/historical/monthly/{prov}'
    
    file_start = []
    for stat_adjust in np.arange(-5,6):
        file_start.append( f'AHCCD_hist_monthly_{prov}_{station+stat_adjust}')
    

    def listFD(url,file_start=''):
        page = requests.get(url).text

        soup = BeautifulSoup(page, 'html.parser')
        return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').startswith(file_start)]
    
    file_list = []
    for file_s in file_start:
        file_list.extend(listFD(url,file_s))


    first = True

    for file in file_list:


        if first:
            raw_data = pd.read_json(file)

            data = []
            for row in raw_data.features:
                year = row['properties']['year']
                month = row['properties']['period']
                data.append([datetime.datetime.strptime(f'{year} {month}' , '%Y %b'),float(row['properties']['value'])])

            data_type = raw_data.features[0]['properties']['measure_type']
            df = pd.DataFrame(data,columns =['year',data_type])
            df = df.set_index('year')
            df = df.sort_index()
            first = False
        else:
            raw_data = pd.read_json(file)
            data = []
            for row in raw_data.features:
                year = row['properties']['year']
                month = row['properties']['period']
                data.append([datetime.datetime.strptime(f'{year} {month}' , '%Y %b'),float(row['properties']['value'])])

            data_type = raw_data.features[0]['properties']['measure_type']
            df_temp = pd.DataFrame(data,columns =['year',data_type])
            df_temp = df_temp.set_index('year')
            df_temp = df_temp.sort_index()
            df_temo = df_temp.replace(-9999.9,0)
            df = pd.concat([df,df_temp],axis = 1, join_axes=[df.index])
    
    df = df.replace(-9999.9,np.nan)

    df = df[df.index>=datetime.datetime(1970,1,1)]

    return df



NameError: name 'tone_gen' is not defined

In [2]:
df = grab_data('ON',6016975)

df['snow'].max()


In [233]:
df.head()

,temp_max,temp_mean,temp_min,total_precip,pressure_station,rain,wind_speed,pressure_sea_level,snow
year,,,,,,,,,
1965-01-01,-15.6,-22.3,-28.9,24.6,968.7,0.3,12.8,1018.4,24.3
1965-02-01,-13.7,-20.8,-27.8,91.5,966.0,0.0,14.4,1015.3,91.5
1965-03-01,-5.8,-13.7,-21.5,31.5,974.1,0.0,9.5,1023.0,31.5
1965-04-01,6.8,1.0,-4.7,62.1,969.8,31.5,13.3,1016.9,30.7
1965-05-01,15.2,8.6,1.9,100.9,965.2,95.5,16.2,1011.4,5.4


,temp_max,temp_mean,temp_min,total_precip,pressure_station,rain,wind_speed,pressure_sea_level,snow
year,,,,,,,,,
1970-01-01,-15.1,-20.3,-25.5,29.9,969.0,0.0,10.7,1018.5,29.9
1970-02-01,-12.1,-19.0,-25.9,30.5,969.5,0.0,12.5,1018.8,30.5
1970-03-01,-2.9,-10.2,-17.4,33.8,971.9,0.0,10.1,1020.4,33.8
1970-04-01,5.6,0.3,-5.1,47.0,965.5,31.7,14.9,1012.5,15.4
1970-05-01,13.2,7.2,1.1,59.0,970.5,43.6,15.4,1017.0,15.4
1970-06-01,23.8,17.2,10.5,59.8,966.7,59.8,13.6,1012.2,0.0
1970-07-01,25.0,19.7,14.4,133.5,966.9,133.5,13.6,1012.2,0.0
1970-08-01,22.8,17.6,12.3,69.9,967.3,69.9,13.0,1012.7,0.0
1970-09-01,15.6,11.1,6.6,102.0,965.7,102.0,13.3,1011.7,0.0


# Sound Generation

In [ ]:
pcp generation

In [57]:
s = Server().boot()
s.start()
ind = LinTable([(0,3), (20,40), (300,10), (1000,5), (8191,3)])
m = Metro(0.5).play()
tr = TrigEnv(m, table=ind, dur=0.5)
f = FM(carrier=[251,250], ratio=[.2498,.2503], index=tr, mul=.2).out()

In [138]:
s.stop()

In [217]:
s = Server().boot()
s.start()


lfo = Sine([2,5,100], 0, .1, .1)
sine_loop = SineLoop(freq=[100,500,600], feedback=lfo, mul=.01)


In [63]:
s.stop()

In [184]:
s.stop()

In [3]:
from pyo import *
import time
import numpy as np
import math


def parameter_update_rain(climate_data):
    snds3 = ['snds/alum2.wav']
    tabs = SndTable(snds3)


    beat = Metro(time=float(climate_data), poly=3).play()
    wav = SquareTable()
    t = CosTable([(0,0), (100,1), (2000,.3), (8191,0)])
    amp = TrigEnv(beat, table=tabs, dur=.25, mul=.3)
    pitch = TrigXnoiseMidi(beat,dist = 4,mrange = (28,48))

    return [wav,pitch,amp]

def parameter_update_snow(climate_data):
    
    ind = LinTable([(0,3), (20,40), (300,10), (1000,5), (8191,3)])
    m = Metro(0.9+climate_data).play()
    tr = TrigEnv(m, table=ind, dur=0.5)


    return tr

def parameter_update_wind(climate_data):
    a = Sine(.25, 0, .1, .1)
    
    noise = PinkNoise(a/100+climate_data).mix(2)

    return noise




In [6]:
num = 120
stop = 125

s = Server()
s.boot().start()
snds3 = ['snds/alum2.wav']

tabs = SndTable(snds3)

beat = Metro(time=0.5, poly=5).play()
wav = SquareTable()
t = CosTable([(0,0), (100,1), (2000,.3), (8191,0)])
amp = TrigEnv(beat, table=tabs, dur=.25, mul=.1)
pitch = TrigXnoiseMidi(beat,dist = 2,mrange = (24,48))



rain = Osc(table=wav,freq= pitch ,mul = amp*1.0).out()

'''

tabs = SndTable(snds3)

beat_snow = Metro(time=0.5, poly=5).play()
wav_snow = SquareTable()
t_snow = CosTable([(0,0), (100,1), (2000,.3), (8191,0)])
amp_snow = TrigEnv(beat_snow, table=tabs, dur=.25, mul=.3)
pitch_snow = TrigXnoiseMidi(beat_snow,dist = 4,mrange = (100,100))

snow = Osc(table=wav_snow,freq= pitch_snow ,mul = amp_snow).out()
'''

if 'snow' in df.columns.values:
    snow_diff = float(df['snow'].max()) - float(df['snow'].min())
    snow_data = 1.333*(pd.to_numeric(df['snow']) - float(df['snow'].mean()))/float(snow_diff)







    ind = LinTable([(0,3), (20,40), (300,10), (1000,5), (8191,3)])
    m = Metro(.9).play()
    tr = TrigEnv(m, table=ind, dur=0.5)
    snow = FM(carrier=[251,250], ratio=[.2498,.2503], index=tr, mul=.02).out()




a = Sine(.25, 0, .1, .1)
noise = PinkNoise(a/100+0.01).mix(2)
tone = Tone(noise,1000).out()



if 'temp_mean' in df.columns.values:
    temp_diff = float(df['temp_mean'].max()) - float(df['temp_mean'].min())
    temp_data = 0.2*(pd.to_numeric(df['temp_mean']) - float(df['temp_mean'].mean()))/float(temp_diff)


lfo = Sine([2,5], 0, .1, .1)
sine_loop = Sine(freq=[400], mul=.1).out()
#sine_loop = SineLoop(freq=[600], feedback=lfo, mul=.004).out()





In [248]:
temp_data[0]

-0.08913916786226685

In [7]:
def pcp_temp():
    
    global num
    global stop
    #Update Rain Parameters
    
    x_rain = float(df['rain'].values[num])

    if x_rain <= 0.0:
        value_rain = 0.0
    else:
        value_rain = 1-(float(x_rain)/(1.1*float(df['rain'].max())))

   

    if 0< value_rain < 0.1:
        value_rain = 0.1
    

    new_para = parameter_update_rain(value_rain)

    rain.table = new_para[0]
    rain.freq = new_para[1]
    rain.mul = new_para[2]
    
    
    x_snow = float(snow_data[num])

    if x_snow <= 0.0:
        value_snow = -0.9
    else:
        value_snow = x_snow


    
    snow.index=parameter_update_snow(value_snow)
    

    
    x_wind = float(df['wind_speed'].values[num])
    print(x_wind)
    if x_wind <= 0.0 or math.isnan(df['wind_speed'].values[num]):
        value_wind = 0
    else:
        value_wind = (float(x_wind)/(1.1*float(df['wind_speed'].max())))


    #if  value_wind>1 :
        #value_wind = 0.01
    print(value_wind)
    
    
    tone.setInput(parameter_update_wind(value_wind/30))
    
    
    x_temp = temp_data[num]
    
    print(x_temp)
    if not math.isnan(x_temp):
        sine_loop.mul = float(0.1+x_temp)
        sine_loop.freq = [200+x_temp*2000]
    print(num)
    num+=1
    if num== stop:
        snow.stop()
        tone.stop()
        rain.stop()
        sine_loop.stop()
        time.sleep(1)
        s.stop()

p =Pattern(pcp_temp,1.5).play()

In [12]:
s.stop()

In [62]:
import math

math.isnan(df['wind_speed'].values[num])

True

In [ ]:
from pyo import *
import time
import numpy as np

num = 0


s = Server().boot().start()
snds3 = ['snds/alum2.wav']

tabs = SndTable(snds3)
'''
beat = Metro(time=0.5, poly=3).play()
wav = SquareTable()
t = CosTable([(0,0), (100,1), (2000,.3), (8191,0)])
amp = TrigEnv(beat, table=tabs, dur=.25, mul=.3)
pitch = TrigXnoiseMidi(beat,dist = 4,mrange = (48,48))



a = Osc(table=wav,freq= pitch ,mul = amp).out()
'''

tabs = SndTable(snds3)

beat_snow = Metro(time=0.5, poly=3).play()
wav_snow = SquareTable()
t_snow = CosTable([(0,0), (100,1), (2000,.3), (8191,0)])
amp_snow = TrigEnv(beat_snow, table=tabs, dur=.25, mul=.1)
pitch_snow = TrigXnoiseMidi(beat_snow,dist = 4,mrange = (100,100))



a_snow = Osc(table=wav_snow,freq= pitch_snow ,mul = amp_snow).out()



def pcp_temp():
    
    global num
    '''
    x = df['Rain'].values[num]

    if x != np.nan:
        value = x
    else:
        value = 0.5
    print(1-value)
    snds3 = ['snds/alum1.wav', 'snds/alum2.wav',
        'snds/alum3.wav', 'snds/alum4.wav']
    tabs = SndTable(snds3)


    beat = Metro(time=float(1-value), poly=3).play()
    wav = SquareTable()
    t = CosTable([(0,0), (100,1), (2000,.3), (8191,0)])
    amp = TrigEnv(beat, table=tabs, dur=.25, mul=.3)
    pitch = TrigXnoiseMidi(beat,dist = 4,mrange = (48,48))

    a.table = wav
    a.freq = pitch
    a.mul = amp
    '''
    
    x_snow = df_snow['Snow'].values[num]

    if x_snow != np.nan:
        value_snow = x_snow
    else:
        value_snow = 0.5
    print(1-value_snow)
    snds3 = ['snds/alum1.wav', 'snds/alum2.wav',
        'snds/alum3.wav', 'snds/alum4.wav']
    tabs = SndTable(snds3)

    beat_snow = Metro(time=float(1-value_snow), poly=1).play()
    wav_snow = SquareTable()
    t_snow = CosTable([(0,0), (100,1), (2000,.3), (8191,0)])
    amp_snow = TrigEnv(beat_snow, table=tabs, dur=.25, mul=.1)
    pitch_snow = TrigXnoiseMidi(beat_snow,dist = 1,mrange = (1000,1000))

    a_snow.table = wav_snow
    a_snow.freq = pitch_snow
    a_snow.mul = amp_snow*10
    
    
    num+=1


p =Pattern(pcp_temp,2).play()


In [91]:
s.start()

In [88]:
s.stop()


In [60]:
df_snow

< Instance of Osc class >

In [5]:
from pyo import *
import random
s = Server().boot()
s.start()
t = HarmTable([1,0,.33,0,.2,0,.143,0,.111])
a = Osc(table=t, freq=[250,251], mul=.2).out()
def pat():
    
    f = random.randrange(200, 401, 25)
    print(f)
    a.freq = [f, f+1]
p = Pattern(pat, .125)
p.play()
#s.gui(locals())

< Instance of Pattern class >

In [3]:
s.start()

In [1]:
from pyo import *
import random

s = Server().boot()

# Sound bank
folder = "snds/"
sounds = ["alum1.wav", "alum2.wav", "alum3.wav", "alum4.wav"]

# Creates the left and right players
sfL = SfPlayer(folder+sounds[0], speed=1, mul=0.5).out()
sfR = SfPlayer(folder+sounds[0], speed=1, mul=0.5).out(1)

# Function to choose a new sound and a new speed for the left player
def newL():
    sfL.path = folder + sounds[random.randint(0, 3)]
    sfL.speed = random.uniform(0.75, 1.5)
    sfL.out()                                                                       

# The "end-of-file" signal triggers the function "newL"
tfL = TrigFunc(sfL["trig"], newL)

# Function to choose a new sound and a new speed for the right player
def newR():
    sfR.path = folder + sounds[random.randint(0, 3)]
    sfR.speed = random.uniform(0.75, 1.5)
    sfR.out(1)

# The "end-of-file" signal triggers the function "newR"
tfR = TrigFunc(sfR["trig"], newR)

In [4]:
s.stop()


In [ ]:
from pyo import *
import random

s = Server(duplex=0).boot()

# Compute the duration, in seconds, of one buffer size.
buftime = s.getBufferSize() / s.getSamplingRate()

# Delay parameters
delay_time_l = Sig(0.125)   # Delay time for the left channel delay.
delay_time_l.ctrl()
delay_feed = Sig(0.75)      # Feedback value for both delays.
delay_feed.ctrl()

# Because the right delay gets its input sound from the left delay, while
# it is computed before (to send its output sound to the left delay), it
# will be one buffer size late. To compensate this additional delay on the
# right, we substract one buffer size from the real delay time.
delay_time_r = Sig(delay_time_l, add=-buftime)

# Setup up a soundfile player.
sf = SfPlayer('snds/alum1.wav').stop()

# Send the original sound to both speakers.
sfout = sf.mix(2).out()

# Initialize the right delay with zeros as input because the left delay
# does not exist yet.
right = Delay(Sig(0), delay=delay_time_r).out(1)

# Initialize the left delay with the original mono source and the right
# delay signal (multiplied by the feedback value) as input.
left = Delay(sf + right * delay_feed, delay=delay_time_l).out()

# One issue with recursive cross-delay is if we set the feedback to
# 0, the right delay never gets any signal. To resolve this, we add a
# non-recursive delay, with a gain that is the inverse of the feedback,
# to the right delay input.
original_delayed = Delay(sf, delay_time_l, mul=1-delay_feed)

# Change the right delay input (now that the left delay exists).
right.setInput(original_delayed + left * delay_feed)

def playit():
    "Assign a sound to the player and start playback."
    which = random.randint(1, 4)
    path = "snds/alum%d.wav" % which
    sf.path = path
    sf.play()

# Call the function "playit" every second.
pat = Pattern(playit, 1).play()
#s.gui(locals())

In [ ]:
s.top()

In [ ]:
from pyo import *

# Creates and boots the server.
# The user should send the "start" command from the GUI.
s = Server().boot()
# Drops the gain by 20 dB.
s.amp = 0.1

# Creates a sine wave player.
# The out() method starts the processing
# and sends the signal to the output.
a = Sine().out()
s.gui(locals())

In [3]:
s.stop()

In [1]:
from pyo import *

s = Server().boot()
s.amp = 0.1

# Creates a sine wave as the source to process.
a = Sine().out()

# Passes the sine wave through an harmonizer.
h1 = Harmonizer(a).out()

# Then the harmonized sound through another harmonizer.
h2 = Harmonizer(h1).out()

# And again...
h3 = Harmonizer(h2).out()

# And again...
h4 = Harmonizer(h3).out()

s.start()

In [2]:
h5 = Harmonizer(h4).out()

In [ ]:
from pyo import *

s = Server().boot()

# Sets fundamental frequency.
freq = 187.5

# Impulse train generator.
lfo1 = Sine(.1).range(1, 50)
osc1 = Blit(freq=freq, harms=lfo1, mul=0.3)

# RC circuit.
lfo2 = Sine(.1, mul=0.5, add=0.5)
osc2 = RCOsc(freq=freq, sharp=lfo2, mul=0.3)

# Sine wave oscillator with feedback.
lfo3 = Sine(.1).range(0, .18)
osc3 = SineLoop(freq=freq, feedback=lfo3, mul=0.3)

# Roland JP-8000 Supersaw emulator.
lfo4 = Sine(.1).range(0.1, 0.75)
osc4 = SuperSaw(freq=freq, detune=lfo4, mul=0.3)

# Interpolates between input objects to produce a single output
sel = Selector([osc1, osc2, osc3, osc4]).out()
sel.ctrl(title="Input interpolator (0=Blit, 1=RCOsc, 2=SineLoop, 3=SuperSaw)")

# Displays the waveform of the chosen source
sc = Scope(sel)

# Displays the spectrum contents of the chosen source
sp = Spectrum(sel)



In [4]:
s.start()

In [12]:
s.stop()

In [10]:
from pyo import *

#s = Server().boot()

x = [1, 0.5, 0.25,0.1]
s.start()

In [11]:
amp = Sine(0.2, mul=0.2, add=0.5)


for f in x:
    a = SineLoop(freq=f, feedback=0.2, mul=amp).out()
    
    


In [7]:
from pyo import *

#s = Server().boot()

# Two streams of midi pitches chosen randomly in a predefined list.
# The argument `choice` of Choice object can be a list of lists to
# list-expansion.
mid = Choice(choice=[60,62,63,65,67,69,71,72], freq=[2,3])

# Two small jitters applied on frequency streams.
# Randi interpolates between old and new values.
jit = Randi(min=0.993, max=1.007, freq=[4.3,3.5])

# Converts midi pitches to frequencies and applies the jitters.
fr = MToF(mid, mul=jit)

# Chooses a new feedback value, between 0 and 0.15, every 4 seconds.
fd = Randi(min=0, max=0.15, freq=0.25)

# RandInt generates a pseudo-random integer number between 0 and `max`
# values at a frequency specified by `freq` parameter. It holds the
# value until the next generation.
# Generates an new LFO frequency once per second.
sp = RandInt(max=6, freq=1, add=8)
# Creates an LFO oscillating between 0 and 0.4.
amp = Sine(sp, mul=0.2, add=0.5)

# A simple synth...
a = SineLoop(freq=fr, feedback=fd, mul=amp).out()

In [16]:
from pyo import *

s = Server().boot()

### Oscilloscope ###

# LFO applied to the `chaos` attribute
lfo = Sine(0.2).range(0, 1)

# Rossler attractor
n1 = Rossler(pitch=0.5, chaos=lfo, stereo=True)

# Lorenz attractor
n2 = Lorenz(pitch=0.5, chaos=lfo, stereo=True)

# ChenLee attractor
n3 = ChenLee(pitch=0.5, chaos=lfo, stereo=True)

# Interpolates between input objects to produce a single output
sel = Selector([n1, n2, n3])
sel.ctrl(title="Input interpolator (0=Rossler, 1=Lorenz, 2=ChenLee)")

# Displays the waveform of the chosen attractor
sc = Scope(sel)

### Audio ###

# Lorenz with very low pitch value that acts as a LFO
freq = Lorenz(0.005, chaos=0.7, stereo=True, mul=250, add=500)
a = Sine(freq, mul=0.3).out()

In [19]:
from pyo import *

s = Server().boot()

# White noise
n1 = Noise(0.3)

# Pink noise
n2 = PinkNoise(0.3)

# Brown noise
n3 = BrownNoise(0.3)

# Interpolates between input objects to produce a single output
sel = Selector([n1, n2, n3]).out()
sel.ctrl(title="Input interpolator (0=White, 1=Pink, 2=Brown)")

# Displays the spectrum contents of the chosen source
sp = Spectrum(sel)

In [ ]:
from pyo import *

s = Server().boot()

# FM implements the basic Chowning algorithm
fm1 = FM(carrier=250, ratio=[1.5,1.49], index=10, mul=0.3)
fm1.ctrl()

# CrossFM implements a frequency modulation synthesis where the
# output of both oscillators modulates the frequency of the other one.
fm2 = CrossFM(carrier=250, ratio=[1.5,1.49], ind1=10, ind2=2, mul=0.3)
fm2.ctrl()

# Interpolates between input objects to produce a single output
sel = Selector([fm1, fm2]).out()
sel.ctrl(title="Input interpolator (0=FM, 1=CrossFM)")

# Displays the spectrum contents of the chosen source
sp = Spectrum(sel)



In [2]:
s.stop()

In [1]:
from pyo import *

s = Server().boot()

# Sets fundamental frequency.
freq = 187.5

# LFO applied to the `sharp` attribute
lfo = Sine(.2, mul=0.5, add=0.5)

# Various band-limited waveforms
osc = LFO(freq=freq, sharp=lfo, mul=0.4).out()
osc.ctrl()

s.start()

In [141]:
from pyo import *

# Set this constant to True to loop over preset bank
WITH_PRESET = True

snds3 = ['snds/alum1.wav', 'snds/alum2.wav',
        'snds/alum3.wav', 'snds/alum4.wav']

presets = [[[16, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
            [16, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
            [16, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]],
           [[16, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1],
            [16, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
            [16, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0]],
           [[16, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1],
            [16, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0],
            [16, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0]],
           [[16, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1],
            [16, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0],
            [16, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1]]]

#s = Server(duplex=0).boot()

tm = Xnoise(dist=2, freq=0.1, x1=.5, x2=3, mul=0.25, add=.5)
b = Beat(time=tm*0.5, w1=[90,30,30,20], w2=[30,90,50,40], w3=[0,30,30,40])
if WITH_PRESET:
    b.setPresets(presets)
    b.recall(0)
b.play()

tabs = SndTable(snds3)
out = TrigEnv(b, table=tabs, dur=b['dur']*2, interp=4, mul=b['amp']*0.5)
pan = SPan(out, pan=[.25, .4, .6, .75]).mix(2)
rev = WGVerb(pan, feedback=.65, cutoff=3500, bal=.2).out()

def change():
    b.fill()
    b.new()

def chAndCall(x):
    b.fill()
    b.recall(x)

x = 0
pre = 0
def ch():
    global x, pre
    x += 1
    if (x % 5) == 0:
        if WITH_PRESET:
            pre = (pre + 1) % 3
            chAndCall(pre)
        else:
            change()

tt = TrigFunc(b['end'][0], function=ch)

In [142]:
s.start()


In [143]:
s.stop()


In [62]:
presets[0]

[[16, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
 [16, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
 [16, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]]

In [138]:
from pyo import *
import random

#s = Server(duplex=0).boot()

SCALES = [[0,2,5,7,9,11], [0,2,3,7,8,11], [0,3,5,7,8,10]]

class Melo:

    def __init__(self, amp=.1, speed=1, midirange=(48,84)):
        # table to record new melody fragment
        self.table = NewTable(2)
        # loopseg generation
        self.base_mel = XnoiseMidi(dist=12, freq=8, x1=1, x2=.25, scale=0, mrange=midirange)
        # snap on scale and convert to hertz
        self.base_melo = Snap(self.base_mel, choice=[0,2,4,5,7,9,11], scale=1)
        # record a new fragment every 10 seconds
        self.trig_rec = Metro(time=10).play()
        self.tab_rec = TrigTableRec(self.base_melo, self.trig_rec, self.table)
        # rise amp of the oscillators after the first recording
        self.amp = Counter(self.tab_rec["trig"], min=1, max=2, mul=amp)
        # random speed for the oscillator reading the melody table + portamento
        self.speed = Choice(choice=[.0625,.125,.125,.125,.25,.5], freq=1.0*speed)
        self.freq = Osc(self.table, self.speed*speed)
        self.freq_port = Port(self.freq, risetime=.01, falltime=.01)
        # 8 randis (freq and amp) to create a chorus of oscillators
        self.rnd_chorus = Randi(min=.99, max=1.01, freq=[random.uniform(3,6) for i in range(8)])
        self.rnd_amp = Randi(min=0, max=.15, freq=[random.uniform(.2,.5) for i in range(8)])
        # oscillators...
        self.osc = LFO(self.freq_port*self.rnd_chorus, type=3, sharp=.75,
                       mul=Port(self.amp, mul=self.rnd_amp)).out()

    def setScale(self, scl):
        self.base_melo.choice = scl

def choose_scale():
    scl = random.choice(SCALES)
    for obj in objs:
        obj.setScale(scl)

a = Melo(amp=.3, speed=0.6, midirange=(-24,90))
#b = Melo(amp=.6, speed=0.5, midirange=(48,72))
#c = Melo(amp=1, speed=0.25, midirange=(36,60))
objs = [a,b,c]

pat = Pattern(time=20, function=choose_scale).play()

In [140]:
s.stop()

In [36]:
from pyo import *

s = Server(duplex=0).boot()

#--> set list of frequencies
low_freqs = [midiToHz(m+7) for m in [36,43.01,48,55.01,60]]
mid_freqs = [midiToHz(m+7) for m in [60,62,63.93,65,67.01,69,71,72]]
high_freqs = [midiToHz(m+7) for m in [72,74,75.93,77,79.01]]
freqs = [low_freqs,low_freqs,mid_freqs,mid_freqs,high_freqs]
#<--

#-->
chx = Choice(choice=freqs, freq=[1,2,3,3,4])
port = Port(chx, risetime=.001, falltime=.001)
sines = SineLoop(port, feedback=[.06,.057,.033,.035,.016], mul=[.15,.15,.1,.1,.06])
pan = SPan(sines, pan=[0, 1, .2, .8, .5]).out()

In [25]:
from pyo import *

#s = Server().boot()

scls = [[0,4,7,11,13,17,21,25,29,33],
        [0,7,12,14,17,21,24,29,31,34],
        [0,7,10,12,15,19,20]]

env = CosTable([(0,0),(50,1),(500,.25),(8191,0)])
wav = SquareTable(5)

met = Metro(time=1, poly=1).play()
note = TrigXnoiseMidi(met, dist="loopseg", x1=1, x2=.2, mrange=(48,97))
snp = Snap(note, choice=scls[0], scale=1)

curscl = 0
def changeScl():
    # change the scale for snp.choice argument
    global curscl
    curscl = (curscl + 1) % len(scls)
    snp.choice = scls[curscl]
    print(snp.choice)

metscl = Metro(time=1).play()
tr = TrigFunc(metscl, function=changeScl)

c = TrigEnv(met, table=env, mul=0.5)
d = Osc(table=wav, freq=snp, mul=c).out()
d1 = Osc(table=wav, freq=snp*0.1, mul=c).out()
d2 = Osc(table=wav, freq=snp*1.002, mul=c).out()
d3 = Osc(table=wav, freq=snp*0.997, mul=c).out()

In [4]:
s.start()

In [151]:
s.stop()

In [31]:
d.stop()

< Instance of Osc class >

In [150]:
d3.stop()

< Instance of Osc class >

In [149]:
s = Server().boot()
s.start()
n = Noise(.3)
lf = Sine(freq=.2, mul=800, add=1000)
f = Tone(n, lf).mix(2).out()